In [533]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

Data cleaning is definitely a "practice makes perfect" skill. Using this dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013, determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . You will need to do considerable data cleaning in order to extract accurate estimates, and may want to look into data encoding methods if you get stuck. For a real bonus round, identify the open access prices paid by subject area.

As noted in the previous assignment, don't modify the data directly. Instead, write a cleaning script that will load the raw data and whip it into shape. Jupyter notebooks are a great format for this. Keep a record of your decisions: well-commented code is a must for recording your data cleaning decision-making progress. Submit a link to your script and results below and discuss it with your mentor at your next session.

In [534]:
raw_df = pd.read_csv('/Users/Stephanie/downloads/wellcome/WELLCOME_APCspend2013_forThinkful.csv',
                 encoding = "ISO-8859-1")

First we create a new dataframe where we'll make the changes without editing on our original dataframe

In [535]:
df = raw_df

Let's check how many publishers we have

In [536]:
def publisher_number():
    print("There're {} different publishers in the dataframe".
          format(len(set(df.Publisher)))
          )

In [537]:
publisher_number()
pub_count = df[['Publisher','Article title']].groupby(['Publisher'])['Article title'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
pub_count.set_index('Publisher').head(25)

There're 299 different publishers in the dataframe


,count
Publisher,
Elsevier,387
Public Library of Science,278
Wiley,136
Springer,81
Oxford University Press,77
Wiley-Blackwell,56
OUP,56
ASBMB,46
Nature Publishing Group,45


Let's see how it looks

In [538]:
publishers = list(set(df.Publisher))
sorted(publishers)[:50]

['ACS',
 'ACS (Amercian Chemical Society) Publications',
 'ACS Publications',
 'AGA Institute',
 'AMBSB',
 'AMERICAN CHEMICAL SOCIETY',
 'ASBMB',
 'ASBMB Cadmus',
 'ASBMB/Cadmus',
 'ASBMB/Cenveo Publisher Services',
 'ASBMC /CENVEO',
 'ASM',
 'ASM (American Society for Microbiology)',
 'American Association of Immunologists',
 'American Chemical Society',
 'American Chemical Society Publications',
 'American College of Chest Physicians',
 'American Physiological Society',
 'American Psychiatric Association',
 'American Psychiatric Publishing',
 'American Psychological Association',
 'American Psychological Association ',
 'American Public Health Association',
 'American Soc for Biochemistry and Molecular Biology',
 'American Society for Biochemistry and Molecular Biolgy',
 'American Society for Biochemistry and Molecular Biology',
 'American Society for Biochemistry and Molecular Biology ',
 'American Society for Investigative Pathology',
 'American Society for Microbiology',
 'America

Initially we're going to use lowercase characters for all publishers, remove any white spaces at the start or end of the string and remove any 'the' at the beginning of a name. That will hopefully unify some of the names

In [539]:
df.Publisher = df.Publisher.str.strip()
df.Publisher = df.Publisher.str.lower()
df.Publisher = df.Publisher.apply(lambda x: x[4:] if x[:3]=='the' else x)
publisher_number()

There're 245 different publishers in the dataframe


The list contains also several variations of the same publisher, plus misspellings, use of acronyms etc.

First, we're going to tackle misspellings and little variations, creating a function that assesses how similar to strings are. We're only going to look at words that are long enough to avoid changing acronyms that might look almost the same but refer to actually different things.

We will replace any string that has a ratio of 0.8 similarity or higher

In [540]:
def similar(a, b, ratio):
    return SequenceMatcher(None, a, b).ratio() > ratio

In [541]:
publishers = list(set(df.Publisher))
publishers_checklist = []

for pub in publishers:
    if len(pub) > 9:
        if pub not in publishers_checklist:
            df.Publisher = df.Publisher.apply(lambda x: pub if similar(pub, x, 0.8) == True else x)
            publishers_checklist.append(pub)
publisher_number()

There're 161 different publishers in the dataframe


That got rid of some of the variations, but if we print the list, there's still many repeated names

In [542]:
publishers = list(set(df.Publisher))
sorted(publishers)

['acs',
 'acs (amercian chemical society) publications',
 'aga institute',
 'ambsb',
 'american association of immunologists',
 'american chemical society',
 'american chemical society petroleum research trust',
 'american college of chest physicians',
 'american physiological society',
 'american psychiatric association',
 'american psychiatric publishing',
 'american public health association',
 'american soc for biochemistry and molecular biology',
 'american society for investigative pathology',
 'american society for nutrition',
 'american society of hamatology',
 'american society of human genetics (elsevier)',
 'american society of pediatrics',
 'american speech-language-hearing association',
 'asbmb',
 'asbmb/cadmus',
 'asbmc /cenveo',
 'asm',
 'association for research in vision & ophthalmology',
 'bentham science publishers',
 'berhahn books',
 'biochem journal',
 'biomed central limited',
 'bioscientifica',
 'blackwell publishing ltd/wiley',
 'bmc',
 'bmj',
 'bmj group',
 'b

The next thing will do is to merge all names that share the first two words, unless they start with 'american' as we have a lot of 'american society of...' that don't necessarily refer to the same Publisher

In [543]:
publishers = list(set(df.Publisher))
publishers_checklist = []

for pub in publishers:
    if len(pub.split()) > 1:
        if 'american' not in pub:
            splitted = pub.split()
            words = splitted[0] + ' ' + splitted[1]
            if pub not in publishers_checklist:
                df.Publisher = df.Publisher.apply(
                                lambda x: x.lower() if len(x.split()) > 1 else x)
                df.Publisher = df.Publisher.apply(
                                lambda x: pub if words in x else x)
                publishers_checklist.append(pub)
            
        

In [544]:
publisher_number()

There're 137 different publishers in the dataframe


In [545]:
sorted(set(df.Publisher))

['acs',
 'acs (amercian chemical society) publications',
 'aga institute',
 'ambsb',
 'american association of immunologists',
 'american chemical society',
 'american chemical society petroleum research trust',
 'american college of chest physicians',
 'american physiological society',
 'american psychiatric association',
 'american psychiatric publishing',
 'american public health association',
 'american soc for biochemistry and molecular biology',
 'american society of hamatology',
 'american society of human genetics (elsevier)',
 'american society of pediatrics',
 'american speech-language-hearing association',
 'asbmb',
 'asbmb/cadmus',
 'asbmc /cenveo',
 'asm',
 'association for research in vision & ophthalmology',
 'bentham science publishers',
 'berhahn books',
 'biochem journal',
 'biomed central limited',
 'bioscientifica',
 'blackwell publishing ltd/wiley',
 'bmc',
 'bmj',
 'bmj group',
 'bmj journals',
 'bmj publishing group ltd & british thoracic society',
 'boulevard',


In [546]:
keywords = ['wiley', 'cambridge', 'elsevier',
            'springer', 'wolters', 'bmj',
            'frontiers', 'hindawi', 'oxford',
            'sage', 'nature', 'cadmus',
            'nature', 'plos', 'pnas']

for keyword in keywords:
    df.Publisher = df.Publisher.apply(
                                lambda x: keyword if keyword in x else x)

publisher_number()

There're 107 different publishers in the dataframe


In [547]:
sorted(set(df.Publisher))

['acs',
 'acs (amercian chemical society) publications',
 'aga institute',
 'ambsb',
 'american association of immunologists',
 'american chemical society',
 'american chemical society petroleum research trust',
 'american college of chest physicians',
 'american physiological society',
 'american psychiatric association',
 'american psychiatric publishing',
 'american public health association',
 'american soc for biochemistry and molecular biology',
 'american society of hamatology',
 'american society of pediatrics',
 'american speech-language-hearing association',
 'asbmb',
 'asbmc /cenveo',
 'asm',
 'association for research in vision & ophthalmology',
 'bentham science publishers',
 'berhahn books',
 'biochem journal',
 'biomed central limited',
 'bioscientifica',
 'bmc',
 'bmj',
 'boulevard',
 'brill',
 'british medical journal',
 'byophysical society',
 'cadmus',
 'cambridge',
 'cell press',
 'cenveo publisher services/asm jv1',
 'coaction',
 'cold spring harbor laboratory pres

In [548]:
pub_words = []

publishers = list(set(df.Publisher))
for pub in publishers:
    words = pub.split()
    
    for word in words:
        pub_words.append(word)

pub_words = set(pub_words)
for word in pub_words:
    if word.isalnum() is False:
        print(word)

society)
(amercian
wliey-blackwell
/cenveo
services/asm
public.service.co.uk
speech-language-hearing
&
t&f
inc.
(faseb)
liebert,


In [549]:
replace = {'/': ' ',
           '-': ' ',
           '&': ' and '
           , '(': '',
           ')': ''}

for k, v in replace.items():
    df.Publisher = df.Publisher.str.replace(k,v)

In [550]:
pub_words

{'&',
 '(amercian',
 '(faseb)',
 '/cenveo',
 'academy',
 'access',
 'acs',
 'aga',
 'aids',
 'ambsb',
 'america',
 'american',
 'and',
 'ann',
 'asbmb',
 'asbmc',
 'asm',
 'association',
 'basque',
 'bentham',
 'berhahn',
 'biochem',
 'biochemistry',
 'biologists',
 'biology',
 'biomed',
 'biosciences',
 'bioscientifica',
 'bmc',
 'bmj',
 'books',
 'boulevard',
 'brill',
 'british',
 'byophysical',
 'cadmus',
 'cambridge',
 'cell',
 'center',
 'central',
 'cenveo',
 'chemical',
 'chemistry',
 'chest',
 'clearace',
 'coaction',
 'cold',
 'college',
 'communications',
 'company',
 'copyright',
 'corp',
 'corporation',
 'country',
 'crystallography',
 'cshlp',
 'cup',
 'elseveier',
 'elsevier',
 'endocrine',
 'endocrinolog',
 'european',
 'experimental',
 'experiments',
 'faseb',
 'federation',
 'ferrata',
 'for',
 'foundation',
 'francis',
 'frontiers',
 'future',
 'hamatology',
 'harbor',
 'health',
 'healthcare',
 'hindawi',
 'hopkins',
 'immunologists',
 'impact',
 'in',
 'inc.',
 'in

In [551]:
pub_words = []

publishers = list(set(df.Publisher))
for pub in publishers:
    words = pub.split()
    
    for word in words:
        pub_words.append(word)

pub_words = set(pub_words)

In [552]:
word_pairs = {}

for word in pub_words:
    for word_2 in pub_words:
        if (similar(word, word_2, 0.75) == True
            and (len(word) == len(word_2) or len(word) == len(word_2) + 1 or len(word) == len(word_2) - 1)
            and word != word_2
            ):
            word_pairs[word] = word_2

word_pairs

{'amercian': 'american',
 'america': 'amercian',
 'american': 'amercian',
 'asbmb': 'asbmc',
 'asbmc': 'asbmb',
 'center': 'central',
 'central': 'center',
 'elseveier': 'elsevier',
 'elsevier': 'elseveier',
 'experimental': 'experiments',
 'experiments': 'experimental',
 'journal': 'journals',
 'journals': 'journal',
 'jscimed': 'sciedu',
 'publisher': 'publishers',
 'publishers': 'publisher',
 'sciedu': 'jscimed',
 'science': 'sciences',
 'sciences': 'science',
 'socety': 'society',
 'society': 'socety',
 'wiley': 'wliey',
 'wliey': 'wiley'}

In [553]:
mispelled = ['amercian', 'elseveier', 'socety', 'wliey']
replace = {x:word_pairs[x] for x in mispelled}

In [554]:
for key, value in replace.items():
    df.Publisher = df.Publisher.apply(
                            lambda x: value if key in x else x)

df.Publisher = df.Publisher.apply(lambda x: 'wiley' if 'wl' in x else x)

df.Publisher = df.Publisher.apply(lambda x: 'oxford' if x =='oup'else x)

df.Publisher = df.Publisher.str.capitalize()


publisher_number()

There're 104 different publishers in the dataframe


In [555]:
pub_count_final = df[['Publisher','Article title']].groupby(['Publisher'])['Article title'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
pub_count_final.set_index('Publisher').head(5)

,count
Publisher,
Elsevier,406
Plos,307
Wiley,270
Oxford,167
Springer,95


Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [557]:
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost'}, inplace=True)
df.Cost = df.Cost.str.replace('£','')
df.Cost = df.Cost.str.replace('$','')
df.Cost = df.Cost.apply(lambda x: round(float(x),2))

In [558]:
print(df.groupby('Publisher').mean().head(5))
print(df.groupby('Publisher').median().head(5))
print(df.groupby('Publisher').std().head(5))

                                              Cost
Publisher                                         
Acs                                    1398.371429
Aga institute                           238.080000
Ambsb                                   265.670000
American                               1204.051667
American association of immunologists  2571.540000
                                           Cost
Publisher                                      
Acs                                     685.880
Aga institute                           238.080
Ambsb                                   265.670
American                               1294.685
American association of immunologists  2571.540
                                             Cost
Publisher                                        
Acs                                    918.733083
Aga institute                                 NaN
Ambsb                                         NaN
American                               250.778454
America